In [ ]:
import pyodbc
import requests
import pandas as pd

# Verbindungsstring
conn_str = (
    r'Driver=SQL Server;'
    r'Server=.\SQLEXPRESS;'
    r'Database=studienprojekt;'
    r'Trusted_Connection=yes;'
)

# Verbindung zur Datenbank herstellen
cnxn = pyodbc.connect(conn_str)
cursor = cnxn.cursor()

# Tabelle prüfen
table_name = "stock_data"  # Name der Zieltabelle
start_date = "2019-07-01"  # Startdatum, ab dem Daten überprüft werden sollen

# Daten aus der Tabelle in einen DataFrame laden
select_query = f"SELECT * FROM {table_name}"
df_existing_data = pd.read_sql(select_query, cnxn)

# API-Anfrage durchführen
url = "https://api.polygon.io/v2/aggs/ticker/AAPL/range/1/day/2019-07-01/2021-06-30"
params = {
    "adjusted": "true",
    "sort": "asc",
    "limit": 5000,
    "apiKey": "AtE7dRJG_TP0fPRBCnu1gGpNMe580JCY"
}
response = requests.get(url, params=params)
data = response.json()

# Datenverarbeitung
results = data['results']
df_new_data = pd.DataFrame(results)
df_new_data['timestamp'] = pd.to_datetime(df_new_data['t'], unit='ms')

# Daten nur einfügen, wenn sie noch nicht vorhanden sind
for _, row in df_new_data.iterrows():
    timestamp = row['timestamp']
    existing_row = df_existing_data[df_existing_data['timestamp'] == timestamp]
    if existing_row.empty:
        # Daten in die Tabelle einfügen
        insert_query = f"INSERT INTO {table_name} (timestamp, o, h, l, c, v) VALUES (?, ?, ?, ?, ?, ?)"
        cursor.execute(insert_query, timestamp, row['o'], row['h'], row['l'], row['c'], row['v'])

# Änderungen in der Datenbank übernehmen
cnxn.commit()

print("Daten wurden erfolgreich in die SQL-Tabelle eingefügt.")

In [7]:
import pyodbc
import yfinance as yf
import pandas as pd

# Verbindungsstring
conn_str = (
    r'Driver=SQL Server;'
    r'Server=.\SQLEXPRESS;'
    r'Database=studienprojekt;'
    r'Trusted_Connection=yes;'
)

# Verbindung zur Datenbank herstellen
cnxn = pyodbc.connect(conn_str)
cursor = cnxn.cursor()

# Tabelle prüfen
table_name = "stock_data"  # Name der Zieltabelle
start_date = "2016-01-01"  # Startdatum, ab dem Daten überprüft werden sollen
end_date = "2023-06-18"  # Enddatum, bis zu dem Daten überprüft werden sollen

# Daten aus der Tabelle in einen DataFrame laden
select_query = f"SELECT * FROM {table_name}"
df_existing_data = pd.read_sql(select_query, cnxn)

# Ticker-Symbol definieren
ticker_symbol = "AAPL"

# Daten von yfinance abrufen
stock_data = yf.download(ticker_symbol, start=start_date, end=end_date)

# Datenverarbeitung
stock_data.reset_index(inplace=True)
stock_data.rename(columns={'Date': 'timestamp'}, inplace=True)
stock_data['timestamp'] = pd.to_datetime(stock_data['timestamp'])

# Daten nur einfügen, wenn sie noch nicht vorhanden sind
for _, row in stock_data.iterrows():
    timestamp = row['timestamp']
    existing_row = df_existing_data[df_existing_data['timestamp'] == timestamp]
    if existing_row.empty:
        # Daten in die Tabelle einfügen
        insert_query = f"INSERT INTO {table_name} (timestamp, [open], high, low, [close], volume) VALUES (?, ?, ?, ?, ?, ?)"
        cursor.execute(insert_query, timestamp, row['Open'], row['High'], row['Low'], row['Close'], row['Volume'])

# Änderungen in der Datenbank übernehmen
cnxn.commit()

print("Daten wurden erfolgreich in die SQL-Tabelle eingefügt.")


C:\Users\HerbyHerb\AppData\Local\Temp\ipykernel_19180\2497997119.py:24: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_existing_data = pd.read_sql(select_query, cnxn)


[*********************100%***********************]  1 of 1 completed
Daten wurden erfolgreich in die SQL-Tabelle eingefügt.
